# Adaptive Semantic Communication Analysis

This notebook analyzes the results from the DRL agent simulation.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Set style
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = [12, 6]

## 1. Load Data

In [2]:
import os
import glob

RUNS_ROOT = "runs"

def get_latest_run_dir():
    """Finds the most recent run directory in runs/"""
    if not os.path.exists(RUNS_ROOT):
        return None
    
    # Get all dirs starting with run_
    run_dirs = glob.glob(os.path.join(RUNS_ROOT, "run_*"))
    if not run_dirs:
        # Fallback to current dir if no run_ folders
        if os.path.exists(os.path.join(RUNS_ROOT, "results.csv")):
            return RUNS_ROOT
        return None
    
    # Sort by name (timestamp)
    latest_run = sorted(run_dirs)[-1]
    return latest_run

latest_dir = get_latest_run_dir()
if latest_dir:
    csv_path = os.path.join(latest_dir, "results.csv")
    print(f"Analyzing latest run: {latest_dir}")
    try:
        df = pd.read_csv(csv_path)
        print(f"Loaded {len(df)} steps.")
        display(df.tail())
    except FileNotFoundError:
        print(f"File not found: {csv_path}. Please run the simulation first.")
        df = pd.DataFrame()
else:
    print("No run directories found in runs/")
    df = pd.DataFrame()

## 2. Reward Analysis

In [ ]:
if not df.empty:
    # Calculate Moving Average
    window_size = 50
    df['reward_ma'] = df['reward'].rolling(window=window_size).mean()

    # 1. Main Reward Plot with Actions Colored
    plt.figure(figsize=(14, 8))
    sns.scatterplot(data=df, x='step', y='reward', hue='action', alpha=0.6, palette='viridis', s=15)
    sns.lineplot(data=df, x='step', y='reward_ma', label=f'{window_size}-Step MA', color='red', linewidth=3)
    plt.title("Reward Evolution by Action")
    plt.ylabel("Reward")
    plt.xlabel("Step")
    plt.ylim(-60, 5)
    plt.grid(True, alpha=0.3)
    plt.legend(title='Action')
    plt.show()
    
    # 2. Zoomed View (Stable Performance)
    plt.figure(figsize=(14, 6))
    zoom_df = df[df['reward'] > -15] 
    if not zoom_df.empty:
        sns.scatterplot(data=zoom_df, x='step', y='reward', hue='action', alpha=0.5, palette='viridis', s=15)
        sns.lineplot(data=df, x='step', y='reward_ma', color='red', linewidth=2, label='Trend')
        plt.title("Reward Detail (Zoomed: > -15)")
        plt.ylim(-15, 2)
        plt.grid(True, alpha=0.3)
        plt.show()

    # 3. Focused Moving Average (-4 to 0) - Requested by User
    plt.figure(figsize=(14, 6))
    # Plot faint background points
    sns.scatterplot(data=df, x='step', y='reward', hue='action', alpha=0.1, palette='viridis', s=10, legend=False)
    # Plot strong trend line
    sns.lineplot(data=df, x='step', y='reward_ma', color='blue', linewidth=3, label=f'{window_size}-Step MA')
    plt.title("Reward Moving Average (Focused View: -4 to 0)")
    plt.ylabel("Reward")
    plt.xlabel("Step")
    plt.ylim(-4, 0.5) 
    plt.grid(True, alpha=0.5)
    plt.legend()
    plt.show()

## 3. Action Distribution over Time

In [4]:
if not df.empty:
    # Convert action strings to categorical if needed, or mapped
    # Assumes action column contains strings like 'SEM_LOCAL', 'RAW'
    
    # Rolling count of actions
    action_dummies = pd.get_dummies(df['action'])
    rolling_actions = action_dummies.rolling(window=100).mean()
    
    rolling_actions.plot.area(alpha=0.6)
    plt.title("Action Distribution (100-step Rolling Average)")
    plt.ylabel("Proportion")
    plt.xlabel("Step")
    plt.legend(title='Action')
    plt.show()

## 4. Policy Behavior: Impact of Network Conditions

In [5]:
if not df.empty:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Noise vs Action
    sns.scatterplot(data=df, x='noise', y='action', hue='action', ax=ax1, alpha=0.5)
    ax1.set_title("Action Choice vs Network Noise")
    
    # Bandwidth vs Action
    sns.scatterplot(data=df, x='bandwidth', y='action', hue='action', ax=ax2, alpha=0.5)
    ax2.set_title("Action Choice vs Bandwidth")
    
    plt.tight_layout()
    plt.show()

## 5. Reward Components Analysis (Latency, Error/MSE, Data Size)
Detailed view of the individual components that make up the reward over time.

In [ ]:
if not df.empty:
    # Calculate Moving Averages for clearer trends
    window = 50
    df['mse_ma'] = df['mse'].rolling(window=window).mean()
    df['latency_ma'] = df['latency'].rolling(window=window).mean()
    df['data_size_ma'] = df['data_size'].rolling(window=window).mean()

    fig, axes = plt.subplots(4, 1, figsize=(14, 20), sharex=True)
    
    # 1. Error (MSE) Plot
    sns.scatterplot(data=df, x='step', y='mse', hue='action', alpha=0.3, palette='viridis', s=10, ax=axes[0], legend=False)
    sns.lineplot(data=df, x='step', y='mse_ma', color='red', label=f'{window}-Step MA', ax=axes[0])
    axes[0].set_title('Reconstruction Error (MSE) over Time')
    axes[0].set_ylabel('MSE Loss')
    axes[0].grid(True, alpha=0.3)
    
    # 2. Latency Plot
    sns.scatterplot(data=df, x='step', y='latency', hue='action', alpha=0.3, palette='viridis', s=10, ax=axes[1], legend=True)
    sns.lineplot(data=df, x='step', y='latency_ma', color='red', label=f'{window}-Step MA', ax=axes[1])
    axes[1].axhline(y=1.0, color='black', linestyle='--', label='Deadline (1.0s)')
    axes[1].set_title('End-to-End Latency over Time')
    axes[1].set_ylabel('Latency (s)')
    axes[1].grid(True, alpha=0.3)
    
    # 3. Data Size Plot
    sns.scatterplot(data=df, x='step', y='data_size', hue='action', alpha=0.3, palette='viridis', s=10, ax=axes[2], legend=False)
    sns.lineplot(data=df, x='step', y='data_size_ma', color='red', label=f'{window}-Step MA', ax=axes[2])
    axes[2].set_title('Transmitted Data Size over Time')
    axes[2].set_ylabel('Data Size (Bytes)')
    axes[2].grid(True, alpha=0.3)

    # 4. Bandwidth vs Latency Dynamics (Dual Axis)
    ax4 = axes[3]
    ax4_twin = ax4.twinx()
    sns.lineplot(data=df, x='step', y='bandwidth', color='blue', alpha=0.6, label='Bandwidth', ax=ax4)
    sns.lineplot(data=df, x='step', y='latency_ma', color='red', alpha=0.8, label='Latency MA', ax=ax4_twin)
    ax4.set_title('Bandwidth & Latency Dynamics')
    ax4.set_ylabel('Bandwidth (Mbps)', color='blue')
    ax4_twin.set_ylabel('Latency (s)', color='red')
    ax4.set_xlabel('Step')
    ax4.grid(True, alpha=0.3)
    
    # Ask matplotlib to collect lines and labels from both axes for the legend
    lines, labels = ax4.get_legend_handles_labels()
    lines2, labels2 = ax4_twin.get_legend_handles_labels()
    ax4.legend(lines + lines2, labels + labels2, loc='upper right')
    if ax4_twin.get_legend():
        ax4_twin.get_legend().remove()
    
    plt.tight_layout()
    plt.show()
